In [1]:
import os
import sys
import numpy as np
from scipy.io import loadmat
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from astropy.io import fits
from astropy.table import Table
from astropy.cosmology import WMAP9 as cosmo
%matplotlib inline

In [2]:
ngal=500
seed=123
rand = np.random.RandomState(seed)

In [3]:
# tts is a MxN matrix where M is the number of days for which there are templates and N is the wavelength range
# wws gives the wavelength range
hsiao = fits.open('/home/orgraur/programs/desi/spectro/templates/basis_templates/v2.1/sne_templates_v1.0.fits')
tts = hsiao[0].data
wws = np.arange(100,25000,10)

In [4]:
# SN Ia LF from Richardson et al. (2014), Table 1
M_B = -19.25 
M_sig = 0.5 
alpha = 1.52

In [5]:
# Johnson B-band light curve template
#   (0) - epoch relative to B-band maximum [day]
#   (1) - B mag [AB]
#   (2) - SDSS r' mag
#   (3) - DECAM_2014 r mag
#   (4) - DECAM_2014 z mag
lc_data = loadmat('trunk/light_curve_B_normalized_DESI.mat' )
lc = lc_data['lc']
days = lc_data['lc'][:,0]
L0 = lc_data['lc'][:,1]

In [6]:
# load the color correction vector (going from R to DECAM r', for BSG, or DECAM z', for LRG)
survey = 1 # let's go with BGS as the default
if survey == 1:
    color = loadmat('/home/orgraur/matlab/desi/decam_r_color.mat')
else:
    color = loadmat('/home/orgraur/matlab/desi/decam_z_color.mat')
color = color['color'][:,0]

In [7]:
# choose which SN templates to use
tempint = 0 # easy way - use integer template IDs and phases
if tempint == 0:
    # random SN template IDs
    sntemp = rand.randint(0, len(days), size=ngal)
    phase = days[sntemp] # the phases of those SN templates
else:
    # Choose the phase with uniform probability between the min/max (ignoring template ID).
    phase = rand.uniform(days.min(), days.max(), size=ngal)
    # this is where we would interpolate between templates... not necessary for now

In [8]:
redshift = rand.uniform(0.01, 0.4, size=ngal) # uniform distribution in redshift 0<z<0.4
rgal = rand.normal(19, 1.8, size=ngal) # uniform distribution in galaxy r-band mag 12<r<22

In [9]:
# calculate the distance modulii of the redshifts generated above
dm = cosmo.distmod(redshift)

In [10]:
# generate random stretch values

# stretch vs epoch relation
smag = loadmat('trunk/stretch_mag.mat')

m = rand.normal(0, M_sig, size=ngal)

s = -0.6579*m +1
x = np.where(np.logical_or(s<0.6, s>1.4))[0]
#print(m[x])
while len(x) != 0:
    x = np.where(np.logical_or(s<0.6, s>1.4))[0]
    m[x] = rand.normal(0, M_sig, size=len(x))
    s[x] = -0.6579*m[x] +1
    
M = M_B + m # randomize the absolute magnitudes

In [12]:
factor = np.zeros(ngal)
for i in range(0, ngal):
    f_host = 10**(-0.4*(rgal[i]+48.6)) # host-galaxy r'-band mag
    
    # stretch the fiducial SN Ia light curve and get the SN Ia mag
    lightcurves = [lc[:,0]*s[i], lc[:,1]-alpha*(s[i]-1)]
    fun = interpolate.interp1d(lightcurves[0], lightcurves[1], fill_value='extrapolate')
    LC_interp = fun(days)
    Norm=-L0+LC_interp
    
    fun1 = interpolate.interp1d(lightcurves[0],Norm, fill_value='extrapolate')
    fun2 = interpolate.interp1d(days,color, fill_value='extrapolate')
    
    mag_sn = M[i] + dm[i].value + fun1(phase[i]) + fun2(phase[i])
    
    f_sn = 10**(-0.4*(mag_sn+48.6))
    factor[i] = f_sn/f_host # the flux factor between the SN and host galaxy

In [ ]:
fig = plt.hist(factor)